In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
sns.set()

In [2]:
raw = pd.read_csv('Bank_data.csv')
raw.head()

,Unnamed: 0,interest_rate,credit,march,may,previous,duration,y
0,0,1.334,0.0,1.0,0.0,0.0,117.0,no
1,1,0.767,0.0,0.0,2.0,1.0,274.0,yes
2,2,4.858,0.0,1.0,0.0,0.0,167.0,no
3,3,4.120,0.0,0.0,0.0,0.0,686.0,yes
4,4,4.856,0.0,1.0,0.0,0.0,157.0,no


In [3]:
data = raw.copy()
data['y'] = data['y'].map({'no':0,'yes':1})
data = data.drop('Unnamed: 0', axis=1)
data.head()

,interest_rate,credit,march,may,previous,duration,y
0,1.334,0.0,1.0,0.0,0.0,117.0,0
1,0.767,0.0,0.0,2.0,1.0,274.0,1
2,4.858,0.0,1.0,0.0,0.0,167.0,0
3,4.120,0.0,0.0,0.0,0.0,686.0,1
4,4.856,0.0,1.0,0.0,0.0,157.0,0


In [4]:
data.describe(include = 'all')

,interest_rate,credit,march,may,previous,duration,y
count,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000
mean,2.835776,0.034749,0.266409,0.388031,0.127413,382.177606,0.500000
std,1.876903,0.183321,0.442508,0.814527,0.333758,344.295990,0.500483
min,0.635000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000
25%,1.042750,0.000000,0.000000,0.000000,0.000000,155.000000,0.000000
50%,1.466000,0.000000,0.000000,0.000000,0.000000,266.500000,0.500000
75%,4.956500,0.000000,1.000000,0.000000,0.000000,482.750000,1.000000
max,4.970000,1.000000,1.000000,5.000000,1.000000,2653.000000,1.000000


In [5]:
y = data['y']
x1 = data[['duration','interest_rate','march','previous']]

In [6]:
x = sm.add_constant(x1)
log = sm.Logit(y,x).fit()
log.summary()

Optimization terminated successfully.
         Current function value: 0.344036
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  518
Model:                          Logit   Df Residuals:                      513
Method:                           MLE   Df Model:                            4
Date:                Sat, 01 Aug 2020   Pseudo R-squ.:                  0.5037
Time:                        22:46:51   Log-Likelihood:                -178.21
converged:                       True   LL-Null:                       -359.05
Covariance Type:            nonrobust   LLR p-value:                 5.272e-77
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.1941      0.303      0.640      0.522      -0.400       0.788
duration          0.0068      0.001      9.371      0.000       0.005       0.008
interest_rate    -0.8213      0.089     -9.248      0.000      -0.995      -0.647
march            -1.9398      0.328     -5.913      0.000      -2.583      -1.297
previous          1.4904      0.499      2.989      0.003       0.513       2.468
=================================================================================
"""

In [7]:
np.exp(0.0068)

1.0068231724945438

Шансы длительности близки к 1. Это говорит нам о том, что хотя длительность значительный предиктор, что шанс в один день повлияет на регрессиию незначительно.  

#### Проверка точности

In [8]:
log.pred_table()

array([[218.,  41.],
       [ 28., 231.]])

In [9]:
df = pd.DataFrame(log.pred_table(),
                  index =  [['Accept','Reject']],
                  columns = [['True 1','True 0']])
df

,True 1,True 0
Accept,218.0,41.0
Reject,28.0,231.0


In [10]:
cf = np.array(df)
coef = (cf[0,0] + cf[1,1])/cf.sum()
coef.round(3)

0.867

#### Teст и оценка модели

In [11]:
test = pd.read_csv('Bank_data_testing.csv')
test.head()

,Unnamed: 0,interest_rate,credit,march,may,previous,duration,y
0,0,1.313,0.0,1.0,0.0,0.0,487.0,no
1,1,4.961,0.0,0.0,0.0,0.0,132.0,no
2,2,4.856,0.0,1.0,0.0,0.0,92.0,no
3,3,4.120,0.0,0.0,0.0,0.0,1468.0,yes
4,4,4.963,0.0,0.0,0.0,0.0,36.0,no


In [12]:
test['y'] = test['y'].map({'no':0,'yes':1})
test = test.drop(['Unnamed: 0'], axis=1)
test = test.drop(['may'], axis=1)
test = test.drop(['credit'], axis=1)
test.head()

,interest_rate,march,previous,duration,y
0,1.313,1.0,0.0,487.0,0
1,4.961,0.0,0.0,132.0,0
2,4.856,1.0,0.0,92.0,0
3,4.120,0.0,0.0,1468.0,1
4,4.963,0.0,0.0,36.0,0


In [13]:
x

,const,duration,interest_rate,march,previous
0,1.0,117.0,1.334,1.0,0.0
1,1.0,274.0,0.767,0.0,1.0
2,1.0,167.0,4.858,1.0,0.0
3,1.0,686.0,4.120,0.0,0.0
4,1.0,157.0,4.856,1.0,0.0
...,...,...,...,...,...
513,1.0,204.0,1.334,1.0,0.0
514,1.0,806.0,0.861,0.0,1.0
515,1.0,290.0,0.879,0.0,0.0
516,1.0,473.0,0.877,0.0,1.0


In [14]:
test_act = test['y']
test_data = test.drop(['y'], axis=1)
test_data = sm.add_constant(test_data)
test_data = test_data[x.columns.values]
test_data

,const,duration,interest_rate,march,previous
0,1.0,487.0,1.313,1.0,0.0
1,1.0,132.0,4.961,0.0,0.0
2,1.0,92.0,4.856,1.0,0.0
3,1.0,1468.0,4.120,0.0,0.0
4,1.0,36.0,4.963,0.0,0.0
...,...,...,...,...,...
217,1.0,458.0,4.963,0.0,0.0
218,1.0,397.0,1.264,1.0,0.0
219,1.0,34.0,1.281,1.0,0.0
220,1.0,233.0,0.739,0.0,0.0


In [15]:
def confusion_matrix(data, actual_values, model):
    pred_values = model.predict(data)
    bins = np.array([0,0.5,1])
    cm = np.histogram2d(actual_values, pred_values,bins = bins)[0]
    accuracy = (cm[0,0]+cm[1,1])/cm.sum()
    return cm, accuracy

In [16]:
cm = confusion_matrix(test_data, test_act, log)
cm

(array([[92., 19.],
        [12., 99.]]),
 0.8603603603603603)

In [17]:
cm_df = pd.DataFrame(cm[0])
cm_df.columns = ['Predicted 0', 'Predicted  1']
cm_df = cm_df.rename (index = {0:'True 0', 1:'True 1'})
cm_df

,Predicted 0,Predicted 1
True 0,92.0,19.0
True 1,12.0,99.0


In [18]:
print (f'Точность модели: {cm[1].round(3)}')

Точность модели: 0.86
